In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import logging
import glob
import json

from utils.ingest import ingest_document
from utils.database_utils import generate_database_and_retriever, populate_database
from utils.summarize import summarize_objects


/Users/robertplanas/Documents/GitHub/kg-augmented-multimodal-rag/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
LOGGER = logging.getLogger(__name__)

In [4]:
folder = "./documents"
assert folder is not None and folder != "", "Folder needs to be specified"
LOGGER.info("Parsing all pdf documents in {}".format(folder))
all_documents = glob.glob(f"{folder}/*.pdf")

In [5]:
all_texts = []
all_tables = []
all_images = []

for doc in all_documents:
    LOGGER.info("Parsing document: {}".format(doc))
    text_objs, table_objs, images_objs = ingest_document(doc)
    all_texts.extend(text_objs)
    all_tables.extend(table_objs)
    all_images.extend(images_objs)

In [6]:
all_texts, all_images, all_tables = summarize_objects(all_texts, all_images, all_tables)

100%|██████████| 2/2 [00:12<00:00,  6.35s/it]


In [7]:
data_base = "./localdb"
retriever = generate_database_and_retriever(main_folder=data_base)
retriever = populate_database(retriever, all_texts, all_images, all_tables)

##### Get all documents in the docstore

In [12]:
all_keys = list(retriever.docstore.yield_keys())
all_documents = retriever.docstore.mget(all_keys)
docutments_dic = {all_keys[i]: all_documents[i] for i in range(len(all_keys))}
## Maybe you should implement by batch size


In [13]:
from utils.knowledge_graph import convert_to_graph_elements_pipeline
import asyncio

In [14]:
final_data = await convert_to_graph_elements_pipeline(docutments_dic, max_concurrency=1)

100%|██████████| 9/9 [07:16<00:00, 48.48s/it]


In [15]:
final_data

[[Relationship(head='análisis', head_type='WORK_OF_ART', relation='PROVIDE_QUALITATIVE_UNDERSTANDING_OF', tail='errores de clasificación del modelo', tail_type='EVENT', confidence=1.0, context='Los resultados del análisis anterior proporcionan una comprensión cualitativa de los errores de clasificación del modelo.', language='es'),
  Relationship(head='predicciones', head_type='ACTIVITY', relation='QUANTIFY_HOW', tail='casos', tail_type='OBJECT', confidence=1.0, context='Para cuantificar cómo se realizan las predicciones y, en consecuencia, identificar los casos en los que el modelo podría equivocarse...', language='es'),
  Relationship(head='modelo', head_type='TECH', relation='TRAINED_USING_METHODOLOGY', tail='sección 4.3.3', tail_type='WORK_OF_ART', confidence=1.0, context='se ha seguido la metodología descrita en la sección 4.3.3 para entrenar dos modelos subrogados interpretables.', language='es'),
  Relationship(head='modelo de regresión lineal', head_type='TECH', relation='IS_SU